In [1]:
from google_agent import Google_agent
from deep_research import Deep_research_engine
from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from pydantic_ai.messages import ModelMessage
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dotenv import load_dotenv
from dataclasses import dataclass
from datetime import datetime
from pydantic import Field
from google.oauth2.credentials import Credentials
from langchain_google_genai import ChatGoogleGenerativeAI
import nest_asyncio
nest_asyncio.apply()
from langchain_openai import ChatOpenAI

import os
from tavily import TavilyClient
load_dotenv()
google_api_key=os.getenv('google_api_key')
tavily_key=os.getenv('tavily_key')
pse=os.getenv('pse')
openai_api_key=os.getenv('openai_api_key')
tavily_client = TavilyClient(api_key=tavily_key)
composio_api_key=os.getenv('composio_api_key')
pydantic_llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))
GEMINI_MODEL='gemini-2.0-flash'
langchain_llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model=GEMINI_MODEL, temperature=0.3)


In [2]:

api_keys={
    'google_api_key':google_api_key,
    'tavily_key':tavily_key,
    'pse':pse,
    'openai_api_key':openai_api_key,
    'composio_key':composio_api_key
}


In [3]:
@dataclass
class Message_state:
    messages: list[ModelMessage]

@dataclass
class Deps:
    deep_research_output: dict
    google_agent_output: dict

In [4]:
class Cortana_agent:
    def __init__(self, api_keys:dict):
        """
        Args:
            
            api_keys (dict): The API keys to use as a dictionary
        """

        GEMINI_MODEL='gemini-2.0-flash'
        
        pydantic_llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=api_keys['google_api_key']))
        openai_llm=ChatOpenAI(api_key=api_keys['openai_api_key'])
        langchain_llm = ChatGoogleGenerativeAI(google_api_key=api_keys['google_api_key'], model=GEMINI_MODEL, temperature=0.3)
        # tools
        llms={'pydantic_llm':pydantic_llm,
              'langchain_llm':langchain_llm,
              'openai_llm':openai_llm}
        deep_research_engine=Deep_research_engine(pydantic_llm,api_keys)
        
        def google_agent_tool(ctx:RunContext[Deps],query:str):
            """
            Use this tool to interact with the google agent, which can, search for images, manage the user's emails, google tasks, contacts, and more.
            Args:
                query (str): The entire query related to the google agent and its capabilities
            Returns:
                str: The response from the google agent 
            """
            google_agent=Google_agent(llms,api_keys)
           
            res=google_agent.chat(query)
            ctx.deps.google_agent_output=res.node_messages
            return res.node_messages[-1]
        


        def search_and_question_answering_tool(ctx: RunContext[Deps], query:str):
            """
            Use this tool to do a deep research on a topic, to gather detailed informations and data, answer_questions from the deep research results or do a quick research if the answer is not related to the deep research.
            Args:
                query (str): The query related to the search_and_question_answering_tool and its capabilities
                

            Returns:
                str: The response from the search_and_question_answering_tool
            """
            @dataclass
            class Route:
                answer: str = Field(default_factory=None,description="the answer to the question if the question is related to the deep research")
                route: str = Field(description="the route, either deep_research or answer_question, or quick_research")
            agent=Agent(pydantic_llm, output_type=Route, instructions="you are a router/question answering agent, you are given a query and you need to decide what to do based on the information provided")
            response= agent.run_sync(f"based on the query: {query}, and the information provided: {ctx.deps.deep_research_output if ctx.deps.deep_research_output else ''} either answer the question or if the answer is not related to the information provided or need more information return 'quick_research' or 'deep_research'")
            route=response.output.route
            if route=='deep_research':
                response=deep_research_engine.chat(query)
                ctx.deps.deep_research_output=response
                return response
            elif route=='answer_question':
                return response.output.answer
            elif route=='quick_research':
                quick_research_agent=Agent(pydantic_llm, tools=[tavily_search_tool(api_keys['tavily_key'])], instructions="do a websearch based on the query")
                result= quick_research_agent.run_sync(query)
                return result.output

        def get_current_time_tool():
            """
            Use this tool to get the current time.
            Returns:
                str: The current time in a formatted string
            """
        
            return f"The current time is {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        @dataclass
        class Cortana_output:
            ui_version: str= Field(description='a markdown format version of the answer for displays if necessary')
            voice_version: str = Field(description='a conversationnal version of the answer for text to voice')
        self.agent=Agent(pydantic_llm, output_type=Cortana_output, tools=[google_agent_tool, search_and_question_answering_tool, get_current_time_tool], system_prompt="you are Cortana, a helpful assistant that can help with a wide range of tasks,\
                          you can use the tools provided to you to help the user with their queries, ask how you can help the user")
        self.memory=Message_state(messages=[])
        self.deps=Deps(deep_research_output={}, google_agent_output={})
    def chat(self, query:str):
        result=self.agent.run_sync(query, deps=self.deps, message_history=self.memory.messages)
        self.memory.messages=result.all_messages()
        return result.output


In [5]:
pydantic_llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=api_keys['google_api_key']))
openai_llm=ChatOpenAI(api_key=api_keys['openai_api_key'])
langchain_llm = ChatGoogleGenerativeAI(google_api_key=api_keys['google_api_key'], model=GEMINI_MODEL, temperature=0.3)
# tools
llms={'pydantic_llm':pydantic_llm,
        'langchain_llm':langchain_llm,
        'openai_llm':openai_llm}

In [6]:
goog=Google_agent(llms,api_keys)

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Cortana\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions

Give Feedback / Get Help:
    On GitHub: https://github.com/ComposioHQ/composio/issues/new
    On Discord: https://dub.composio.dev/discord
    On Email: tech@composio.dev
    Talk to us on Intercom: https://composio.dev
    Book a call with us: https://composio.dev/redirect?url=https://calendly.com/composiohq/support?utm_source=py-sdk-logs&utm_campaign=calendly
If you need to debug this error, set `COMPOSIO_LOGGING_LEVEL=debug`.
  warnings.warn(
c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Cortana\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use

In [9]:
res=goog.chat('search for bars in east village nyc')

In [12]:
res.node_messages[-1]

{'maps_manager': {'data': {'places': [{'displayName': {'languageCode': 'en',
      'text': 'McSorley’s Old Ale House'}},
    {'displayName': {'languageCode': 'en', 'text': 'Double Chicken Please'}},
    {'displayName': {'languageCode': 'en', 'text': 'The Bowery Hotel'}},
    {'displayName': {'languageCode': 'en', 'text': 'Mr. Purple'}},
    {'displayName': {'languageCode': 'en', 'text': 'Book Club Bar'}},
    {'displayName': {'languageCode': 'en', 'text': 'Cafe Mogador'}},
    {'displayName': {'languageCode': 'en', 'text': 'Freemans'}},
    {'displayName': {'languageCode': 'en', 'text': 'Pianos'}},
    {'displayName': {'languageCode': 'en', 'text': 'The Delancey Rooftop'}},
    {'displayName': {'languageCode': 'en', 'text': 'Superbueno'}}]},
  'error': None,
  'successfull': True,
  'successful': True,
  'logId': 'log_e_ptXqf9UXco'}}

In [8]:
res.node_messages[0]

{'list_tools': {'Mail Manager': [{'name': 'GMAIL_DELETE_DRAFT',
    'description': "Delete an email draft using gmail's api."},
   {'name': 'GMAIL_DELETE_MESSAGE',
    'description': "Delete an email message using gmail's api. note: this action requires the integration to have the `https://mail.google.com/` scope."},
   {'name': 'GMAIL_FETCH_EMAILS',
    'description': 'Action to fetch all emails from gmail.'},
   {'name': 'GMAIL_GET_CONTACTS',
    'description': 'Action to get info of contacts saved in google for an authorized account. a custom integration with `https://www.googleapis.com/auth/contacts.readonly` scope is required to use this action.'},
   {'name': 'GMAIL_LIST_DRAFTS',
    'description': "List all email drafts using gmail's api."},
   {'name': 'GMAIL_MOVE_TO_TRASH',
    'description': "Move an email message to trash using gmail's api."},
   {'name': 'GMAIL_REPLY_TO_THREAD',
    'description': 'Action to reply to an email thread in gmail.'},
   {'name': 'GMAIL_SEARCH_PE

In [10]:
message_text=[i['messageText'] for i in goog.state.node_messages[0]['mail_manager']['data']['messages']]

In [12]:
for i in message_text:
    print(i)

Hello there! Welcome to Lesson 8 of the applied LLM evaluations
course (
https://click.convertkit-mail2.com/4zu84ve2x8i7u55wpqztxh64mvm77h5/z2hghnhe7npdl9az/aHR0cHM6Ly93d3cuZXZpZGVudGx5YWkuY29tL2xsbS1ldmFsdWF0aW9uLWNvdXJzZS1wcmFjdGljZQ==
) for AI builders. We continue to evaluate LLM systems on
different tasks, and today, we’ll focus on RAG
(Retrieval-Augmented Generation). We will set up a RAG system and
evaluate its quality.

-------------------------------------
💻 Tutorial on evaluating a RAG system
-------------------------------------

In this tutorial (
https://click.convertkit-mail2.com/4zu84ve2x8i7u55wpqztxh64mvm77h5/p8heh9h4dmg7q3ar/aHR0cHM6Ly93d3cueW91dHViZS5jb20vd2F0Y2g_aW5kZXg9MTAmbGlzdD1QTDlvbVg2aW1wRXVOVHIwS0dMQ2hId2h2Ti1xM1pGMTJkJnY9amNrcDVSMDlBZmc=
), we’ll build and evaluate a RAG system that answers user
queries using the source knowledge base. We will generate a
synthetic dataset to test the system and evaluate its correctness
against the ground truth and faithfulnes

In [5]:
cortana_agent=Cortana_agent(api_keys)

In [ ]:
res=cortana_agent.chat('fetch 2 emails from the inbox')


c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Cortana\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions

Give Feedback / Get Help:
    On GitHub: https://github.com/ComposioHQ/composio/issues/new
    On Discord: https://dub.composio.dev/discord
    On Email: tech@composio.dev
    Talk to us on Intercom: https://composio.dev
    Book a call with us: https://composio.dev/redirect?url=https://calendly.com/composiohq/support?utm_source=py-sdk-logs&utm_campaign=calendly
If you need to debug this error, set `COMPOSIO_LOGGING_LEVEL=debug`.
  warnings.warn(
c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Cortana\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use

ValidationError: 2 validation errors for AIMessage
content.str
  Input should be a valid string [type=string_type, input_value={'role': 'mail_manager', ...d': 'log_a87pPDADzpRM'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
content.list[union[str,dict[any,any]]]
  Input should be a valid list [type=list_type, input_value={'role': 'mail_manager', ...d': 'log_a87pPDADzpRM'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type

In [15]:
print(res.ui_version)

I have fetched 2 emails from your inbox. The first is from LinkedIn News regarding 'Jony Ive's new job at OpenAI' and the second is a job alert for Artificial Intelligence Engineer positions in North America.


In [14]:
print(res.voice_version)

I have fetched 2 emails from your inbox. The first is from LinkedIn News regarding Jony Ive's new job at OpenAI and the second is a job alert for Artificial Intelligence Engineer positions in North America. 


In [7]:
cortana_agent.deps.google_agent_output

{}

In [7]:
cortana_agent.memory.messages

[ModelRequest(parts=[UserPromptPart(content='show 10 emails in the inbox', timestamp=datetime.datetime(2025, 5, 21, 21, 34, 25, 856683, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], instructions='you are Cortana, a helpful assistant that can help with a wide range of tasks,                          you can use the tools provided to you to help the user with their queries, ask how you can help the user', kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='google_agent_tool', args={'query': 'show 10 emails in the inbox'}, tool_call_id='pyd_ai_cc37a375a932459ebbea7ff8071bf003', part_kind='tool-call')], usage=Usage(requests=1, request_tokens=359, response_tokens=14, total_tokens=373, details={}), model_name='gemini-2.0-flash', timestamp=datetime.datetime(2025, 5, 21, 21, 34, 26, 620932, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='google_agent_tool', content='{\'data\': {\'messages\': [{\'attachmentList\': [], \'labe